In [ ]:
import os
import sys

local_path = os.getcwd()
# 设置工作目录为项目的主目录
os.chdir(os.path.join(local_path, "../"))  # 使用相对路径将工作目录切换到 project 文件夹
print("Current working directory:", os.getcwd())
project_path = os.path.abspath(os.path.join(local_path, "../../"))
sys.path.append(project_path)   #将模块查找路径切换

In [ ]:
import torch
import torch.nn as nn
from utils import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
set_seed(42)

In [ ]:
selected_classes = [0, 1]
class_counts = [100, 100]

train_loader = get_dataloader('CIFAR10', batch_size=64, train=True, selected_classes=selected_classes, class_counts=class_counts)
test_loader = get_dataloader('CIFAR10', batch_size=64, train=False)

data_iter = iter(train_loader)
images, labels = next(data_iter)
print(f"Images batch shape: {images.shape}")
print(f"Labels batch shape: {labels.shape}")

batch_size, channel, pixel_size, _ = images.shape

In [ ]:
import train
input_size = channel*pixel_size**2
output_size = 10
lam = 0.1
d_f = 0.2
alpha = 1.0
learning_rate = 0.01
num_epochs = 5
train.train_FBM(input_size, output_size, lam, d_f, alpha, learning_rate, train_loader, num_epochs, device)

# 目的
设置一个深层的FBM网络，通过整体的推理，单独计算每一层的loss，然后逐层反向传播，进行优化。

In [ ]:
# 对该函数功能的实现，并且并且集成在train函数里面

from loss.loss import FBMLoss
import models
import torch.optim as optim
hidden_list = [20, 40, 50]
df_list = [1, 2, 3]
alpha_list = [1, 2, 3]

num = len(hidden_list)
model_list = []
criterion_list = []
optimizer_list = []

lam = 0.1
input_size = channel*pixel_size**2

for output_size, d_f, alpha in zip(hidden_list, df_list, alpha_list):
    model = models.FBMLayer(input_size, output_size).to(device)
    criterion = FBMLoss(output_size, lam, d_f, alpha)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    model_list.append(model)
    criterion_list.append(criterion)
    optimizer_list.append(optimizer)

    input_size = output_size

input_size = channel*pixel_size**2
# 训练模型
for epoch in range(num_epochs):

    for images, labels in train_loader:
        # 将图像和标签移动到 GPU 上
        outputs = images.view(-1, input_size).to(device)  # 展平图像并转移到 GPU
        labels = labels.to(device)  # 标签移动到 GPU

        loss_list = []
        for model, loss, optimizer in zip(model_list, criterion_list, optimizer_list):
            # 前向传播
            inputs = outputs.detach()
            outputs = model(inputs)
            loss = criterion(outputs, labels, model.linear.weight)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_list.append(f'{loss.item():.4f}')
            
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss_list}')

pipeline = models.ModelPipeline()
for model in model_list:
    pipeline.add_model(model)



In [ ]:
# 调用train函数
import train
model, _ = train.pre_DFBM(train_loader, [100, 100, 100], [1, 2, 3], [1, 2, 3], device, train_MLP=True)

In [ ]:
# 准确率计数
correct = 0
total = 0

# 禁用梯度计算，加速测试过程
with torch.no_grad():
    for images, labels in test_loader:
        # 将数据加载到 GPU
        images = images.view(-1, input_size).to(device)
        labels = labels.to(device)

        # 前向传播
        outputs = model(images)
        
        # 获取预测结果
        _, predicted = torch.max(outputs, 1)
        
        # 更新计数
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 计算准确率
accuracy = 100 * correct / total
print(f'Accuracy on the test dataset: {accuracy:.2f}%')